Details:

http://192.168.1.123:8998  
Livy Server: http://192.168.1.123:8998/ui  
Spark Master: http://192.168.1.123:8080/  
Spark Magic: https://github.com/jupyter-incubator/sparkmagic/blob/master/examples/Magics%20in%20IPython%20Kernel.ipynb 

Let's load up sparkmagic to be able to communicate without spark cluster

In [1]:
%load_ext sparkmagic.magics

Now let's configure our connection

In [ ]:
%manage_spark

Now let's run our scala code

In [ ]:
%%spark

package kMeans

import org.apache.log4j.{Level, Logger}
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.sql.{SQLContext, SparkSession}

object kMeans {

  def main(args: Array[String]): Unit = {

    Logger.getLogger("org").setLevel(Level.OFF)
    Logger.getLogger("akka").setLevel(Level.OFF)

    val spark = SparkSession
      .builder()
      .appName("kMeans")
      .master("local")
      .getOrCreate()

    val sc = spark.sparkContext // doing it this way to be able to import implicits

    val sqlContext = new SQLContext(sc)

    def readExcel(file: String) = sqlContext.read
      .format("com.crealytics.spark.excel")
      .option("location", file)
      .option("useHeader", "true")
      .option("treatEmptyValuesAsNulls", "true")
      .option("inferSchema", "true")
      .option("addColorColumns", "False")
      .load()

    val data = readExcel("C:/Users/jorge.robins/Documents/iris.xlsx")

    // data.show(false)

    val idPointRDD = data.rdd.map(s => Vectors.dense(s.getDouble(0), s.getDouble(1), s.getDouble(2), s.getDouble(3))).cache()

    // pedal width, pedal length, sepal length, sepal width

    // Cluster the data into two classes using KMeans
    val numClusters = 3
    val numIterations = 20
    val clusters = KMeans.train(idPointRDD, numClusters, numIterations)

    // here is what I added to predict data points that are within the clusters

    val WSSSE = clusters.computeCost(idPointRDD)
    println("Within Set Sum of Squared Errors = " + WSSSE)

    //val predictions = clusters.predict(idPointRDD)
    //predictions.collect().foreach(println)

    val vectorsAndCluster = idPointRDD.map { point =>
      val prediction = clusters.predict(point)
      (point.toString, prediction)
    }

In [ ]:
val dataStaging = vectorsAndCluster.map(x=> x._2)
dataStaging.collect().foreach(println)